### Import libraries

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import numpy as np

### Model definition

In [2]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 768 * 16, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Model creation

In [4]:
model = ImageClassifier()

### Data preparation

In [5]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Parcours du dossier "leo" pour obtenir les images et leurs labels
        leo_dir = os.path.join(data_dir, "leo")
        for image_name in os.listdir(leo_dir):
            self.image_paths.append(os.path.join(leo_dir, image_name))
            self.labels.append(0)  # Label 0 pour la classe "leo"

        # Parcours du dossier "orion" pour obtenir les images et leurs labels
        orion_dir = os.path.join(data_dir, "orion")
        for image_name in os.listdir(orion_dir):
            self.image_paths.append(os.path.join(orion_dir, image_name))
            self.labels.append(1)  # Label 1 pour la classe "orion"

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]

        image = Image.open(image_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.image_paths)

### Transform to tensors

In [7]:
# Définir les transformations
transform = transforms.Compose([
    transforms.Resize((2561, 1536)),
    transforms.ToTensor()
])

In [8]:
data_dir = '../constellations/'
train_dataset = CustomDataset(data_dir, transform=transform)
test_dataset = CustomDataset(data_dir, transform=transform)

# Définir les dataloaders pour l'entraînement et les tests
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Optimizer & Loss Function

In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

### Modele Training

In [13]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Mode entraînement
    running_loss = 0.0

    for images, labels in train_dataloader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass et mise à jour des poids
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")

RuntimeError: Numpy is not available